<a href="https://colab.research.google.com/github/dijkstra001/py-for-data/blob/main/py_for_data_case04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Utilização de Python para Análise de Dados: Caso prático - 04**

---

O conteúdo apresentado a seguir tem como objetivo criar cenários práticos e de uso geral onde a linguagem Python pode nos auxiliar de forma rápida e simples na análise de dados.

Esse material tem como objetivo ensinar algumas práticas para facilitar a análise de dados, não focando em técnicas complexas e de uso exclusivo para determinada situação.

O conteúdo tem como base o Livro **Python para Análise de Dados** (Wes McKinney, NOVATEC) e **Data Science do Zero** (Joel Grus, O'Reilly).

### **Realizando operações com as funções map, filter e reduce:**

Map, filter e reduce fazem parte dos métodos que compreendem a **Programação funcional**. A programação funcional é aquela que trata os comandos como funções matemáticas, tendo como um dos seus pilares a **imutabilidade**.

Isso significa que, passando os mesmos argumentos para esses comandos, os resultados sempre serão os mesmos, evitando, dessa forma, resultados indesejados.

De forma resumida, podemos dizer que:

**a) MAP**: a função Map realiza o mapeamento de um conjunto de dados, retornando uma lista com os objetos mapeados de acordo com o método informado, sem alterar a lista original.

**b) FILTER**: a função Filter retorna uma lista de elementos de acordo com o resultado da expressão informada.

**c) REDUCE**: a função Reduce atua como uma função de agregação. Com ela, podemos retornar um único elemento de um conjunto de elementos, de acordo com o método de agregação desejado.



In [1]:
from statistics import mean
from functools import reduce
import operator
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/py-for-data/datasets/case04_dataset.xlsx'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### **Conjunto de dados:**

Nosso conjunto de dados consiste em valores fictícios de empresas, onde temos o faturamento mínimo, máximo, tempo de empresa em anos e quantidade de colaboradores. Como ainda não abordamos extração de dados via web, os valores estarão nesse [arquivo](https://drive.google.com/file/d/1JPYd20BAjFgDyp4eUVNPyAU8sUxWfj-Y/view?usp=sharing).



### **Realizando o processo de análise dos dados**:###

Antes de iniciarmos as aplicações das funções map, filter e reduce, precisamos conhecer nossos dados e identificar se há necessidade de tratamentos. Após os tratamentos, podemos identificar o que podemos transformar com eles, gerando novos dados a partir daqueles que já temos.


**Obs**: Vamos utilizar um novo modelo de conjunto de dados: o formato xlsx.

In [2]:
dataset = pd.read_excel(path)
dataset.head(10)

,empresa,fat_min,fat_max,tempo_mercado_em_anos,colaboradores
0,Empresa1,500,2330000,11,327
1,Empresa2,20500,473717,5,314
2,Empresa3,35120,409153,11,250
3,Empresa4,6221,1050000,5,365
4,Empresa5,3652,101000,5,182
5,Empresa6,4123,163731,4,202
6,Empresa7,26422,1060000,2,323
7,Empresa8,36412,2470000,3,297
8,Empresa9,2548,729203,3,545
9,Empresa10,5563,1520000,9,174


In [3]:
dataset.describe()

,fat_min,fat_max,tempo_mercado_em_anos,colaboradores
count,20.000000,2.000000e+01,20.000000,20.000000
mean,13735.050000,8.762574e+05,7.350000,312.650000
std,12539.926234,7.550121e+05,4.158378,169.477611
min,263.000000,7.116600e+04,2.000000,57.000000
25%,2560.000000,1.860000e+05,4.000000,180.000000
50%,6440.000000,6.559685e+05,6.000000,305.500000
75%,24556.250000,1.280000e+06,11.000000,458.500000
max,36412.000000,2.470000e+06,15.000000,603.000000


### **Criar uma coluna com o valor médio do faturamento:**

Para calcular a média utilizando o map, vamos criar uma função anônima que recebe dois parâmetros e retorna a divisão desses parâmetros. Vamos atribuir esse resultado na variável calcular_media, que terá o comportamento de uma função.

Vamos passar essa função para o primeiro parâmetro do map e os dois parâmetros da função calcular média: fat_min e fat_max. 

O retorno da função map é um objeto map. Por isso, vamos convertê-lo em uma lista para que possamos criar a nova coluna no dataset. Por fim, vamos visualizar os registros incluídos.

In [4]:
fat_min = dataset.fat_min
fat_max = dataset.fat_max

calcular_media = lambda x, y : (x + y) / 2

fat_medio = list(map(calcular_media, fat_min , fat_max))
dataset['fat_medio'] = fat_medio
dataset.head(10)

,empresa,fat_min,fat_max,tempo_mercado_em_anos,colaboradores,fat_medio
0,Empresa1,500,2330000,11,327,1165250.0
1,Empresa2,20500,473717,5,314,247108.5
2,Empresa3,35120,409153,11,250,222136.5
3,Empresa4,6221,1050000,5,365,528110.5
4,Empresa5,3652,101000,5,182,52326.0
5,Empresa6,4123,163731,4,202,83927.0
6,Empresa7,26422,1060000,2,323,543211.0
7,Empresa8,36412,2470000,3,297,1253206.0
8,Empresa9,2548,729203,3,545,365875.5
9,Empresa10,5563,1520000,9,174,762781.5


### **Retornar os valores de acordo com o parâmetro informado:**

Vamos elaborar uma função onde o usuário possa informar os campos da tabela que precisam ser filtrados, além da ordem do filtro (maior ou igual ou menor que o parâmetro informado). De acordo com os parâmetros informados, os valores retornados precisam ser filtrados de acordo com os parâmetros informados.

In [5]:
def filtrar_dataset(dataset, maior_menor:bool, fat_min, fat_max, fat_medio, tempo, colaboradores):
  if maior_menor:
    return dataset.query(f'fat_min >= {fat_min} & fat_max >= {fat_max} & fat_medio >= {fat_medio} & tempo_mercado_em_anos >= {tempo} & colaboradores >= {colaboradores}')
  return dataset.query(f'fat_min <= {fat_min} or fat_max <= {fat_max} or fat_medio <= {fat_medio} or tempo_mercado_em_anos <= {tempo} or colaboradores <= {colaboradores}')
filtrar_dataset(dataset, False, 0, 0, 0, 8, 0)

,empresa,fat_min,fat_max,tempo_mercado_em_anos,colaboradores,fat_medio
1,Empresa2,20500,473717,5,314,247108.5
3,Empresa4,6221,1050000,5,365,528110.5
4,Empresa5,3652,101000,5,182,52326.0
5,Empresa6,4123,163731,4,202,83927.0
6,Empresa7,26422,1060000,2,323,543211.0
7,Empresa8,36412,2470000,3,297,1253206.0
8,Empresa9,2548,729203,3,545,365875.5
11,Empresa12,25563,2090000,4,285,1057781.5
12,Empresa13,2466,1270000,7,136,636233.0
14,Empresa15,1096,71166,3,469,36131.0


### **Aplicando a função filter:**

In [6]:
tempo_mercado_em_anos_filtrado = set(filter(lambda x: x > 3, dataset.tempo_mercado_em_anos.values))
print(f'Tempo de mercado (em anos): {sorted(tempo_mercado_em_anos_filtrado)}\n')

def media_geral_colaboradores():
  return dataset.colaboradores.values.sum() / len(dataset)

colaboradores_filtrado = set(filter(lambda y: y > media_geral_colaboradores(), dataset.colaboradores.values))
print(f'Média de colaboradores geral: {round(media_geral_colaboradores())}\nTotal de colaboradores acima da média de colaboradores geral: {sorted(colaboradores_filtrado)}')

Tempo de mercado (em anos): [4, 5, 7, 9, 10, 11, 12, 13, 14, 15]

Média de colaboradores geral: 313
Total de colaboradores acima da média de colaboradores geral: [314, 323, 327, 365, 455, 469, 495, 545, 592, 603]


### **Aplicando a função reduce:**

In [7]:
# operação de soma:
total_colaboradores = reduce(operator.add, dataset.colaboradores)
print(f'Total colaboradores: {total_colaboradores}')

# operação de subtração:
diff_faturamento = list(reduce(operator.sub, (dataset.fat_max, dataset.fat_min)))
print(f'Diferença faturamento: {sorted(diff_faturamento[0:10])}')

# faturamento médio per capita (fat_medio / nº colaboradores):

def per_capita(x, y):
  return round(x / y, 2)

fat_medio_per_capita = list(reduce(per_capita, (dataset.fat_medio, dataset.colaboradores)))
print(f'Faturamento médio per capita: {sorted(fat_medio_per_capita[0:10])}')

Total colaboradores: 6253
Diferença faturamento: [97348, 159608, 374033, 453217, 726655, 1033578, 1043779, 1514437, 2329500, 2433588]
Faturamento médio per capita: [287.51, 415.48, 671.33, 786.97, 888.55, 1446.88, 1681.77, 3563.46, 4219.55, 4383.8]
